In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
import random
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef, make_scorer
from statistics import stdev, variance, mean
from scipy.interpolate import interp1d
import torch
from torch import nn, optim
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
client_set=set()
others_set=set()
for rec in SeqIO.parse("../fig1/result/drllps_client_clstr_Homo_sapiens.fasta", "fasta"):
    client_set.add(rec.id)
for rec in SeqIO.parse("../fig1/result/drllps_nonllps_clstr_Homo_sapiens.fasta", "fasta"):
    others_set.add(rec.id)
    
mat=np.load("embedding/PTT5XLU50_human.npy", allow_pickle=True)
mat=mat.item()

list_client=[]
list_others=[]
for k in mat.keys():
    if k in others_set:
        list_others.append(mat[k])
    elif k in client_set:
        list_client.append(mat[k])

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, path='checkpoint_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:  
            self.best_score = score   
            self.checkpoint(val_loss, model)  
        elif score <= self.best_score:  
            self.counter += 1   
            if self.verbose:  
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')   
            if self.counter >= self.patience:  
                self.early_stop = True
        else:  
            self.best_score = score  
            self.checkpoint(val_loss, model)  
            self.counter = 0  
            
    def checkpoint(self, val_loss, model):
        if self.verbose:  
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)  
        self.val_loss_min = val_loss  
        
def training_loop(n_epochs, optimizer, model, loss, mask_train, x_train,  y_train):
    loss=loss
    
    n_samples=x_train.shape[0]
    n_val=int(n_samples*0.2)

    shuffled_ind=torch.randperm(n_samples)

    train_ind=shuffled_ind[:-n_val] 
    val_ind=shuffled_ind[-n_val:]
    
    x_val=x_train[val_ind]
    y_val=y_train[val_ind]
    
    x_train=x_train[train_ind]
    y_train=y_train[train_ind]
    
    x_train=x_train
    y_train=y_train
    
    x_val=x_val
    y_val=y_val

    patience=10
    earlystopping = EarlyStopping(patience=patience, verbose=False)
    for epoch in range(1, n_epochs+1):
        model.train()
        
        y_train_pred=model.forward(x_train)
        loss_train=loss(y_train_pred, y_train)
        
        model.eval()
        with torch.no_grad():
            y_val_pred=model.forward(x_val)
            loss_val=loss(y_val_pred, y_val)

        earlystopping(loss_val, model) 
        if earlystopping.early_stop: 
            break
            
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
class FNN2(nn.Module):
    def __init__(self, embeddings_dim=1024, dropout=0.25):
        super(FNN2, self).__init__()

        self.linear = nn.Sequential(
            nn.Linear(embeddings_dim, 32),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(32,2)
        )


    def forward(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        o = self.linear(x)  
        return o
    

class NN(BaseEstimator, ClassifierMixin):
    def __init__(self, n_epochs=500, lr=0.03):
        self.n_epochs = n_epochs
        self.lr = lr
        self.model = None
        self.optim = None
        self.loss = nn.CrossEntropyLoss()

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        X_tensor = torch.tensor(X, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.long)
        n_dim = X_tensor.shape[1]
        self.model=FNN2(embeddings_dim=n_dim)
        self.optim = optim.Adam(self.model.parameters(), lr=self.lr)
        training_loop(
            n_epochs=self.n_epochs,
            optimizer=self.optim,
            model=self.model,
            loss=self.loss,
            mask_train=None,
            x_train=X_tensor,
            y_train=y_tensor,
        )
        return self

    def predict(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float)
            self.model.eval()
            y_pred = self.model(X_tensor)
            _, predicted = torch.max(y_pred, 1)
            return predicted.numpy()

    def predict_proba(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float)
            self.model.eval()
            y_pred = self.model(X_tensor)
            probas = nn.Softmax(dim=1)(y_pred)
            return probas.numpy()

In [ ]:
class Display_auc():
    def __init__(self,x:np.array, y:np.array, model):
        self.x=x
        self.y=y
        self.model=model
        
    def run(self):
        cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
        out_y=[]
        out_y_proba=[]
        scores_rocauc=[]
        scores_prauc=[]
        scores_mcc=[]
        precisions=[]
        recalls=[]
        tprs = []
        mean_fpr = np.linspace(0, 1, 1000)
        
        for train,test in cv.split(self.x, self.y):
            #print(y[train])
            self.model.fit(self.x[train],self.y[train])
            y_pred_proba=self.model.predict_proba(self.x[test])[:,1]
            y_pred=self.model.predict(self.x[test])
            fpr, tpr, _ = roc_curve(self.y[test], y_pred_proba)
            roc_auc=auc(fpr,tpr)
            precision, recall, _ = precision_recall_curve(self.y[test], y_pred_proba)
            pr_auc = auc(recall, precision)
            mcc=matthews_corrcoef(self.y[test], y_pred)
            
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            precisions.append(precision)
            recalls.append(recall)
            scores_rocauc.append(roc_auc)
            scores_prauc.append(pr_auc)
            scores_mcc.append(mcc)
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        self.mean_tpr = mean_tpr
        self.tprs = tprs
        self.precisions = precisions
        self.recalls = recalls
        self.scores_rocauc = scores_rocauc
        self.scores_prauc = scores_prauc
        self.scores_mcc = scores_mcc

In [ ]:
x=np.array(list_client+list_others)
y=np.array([True]*len(list_client) + [False]*len(list_others))
estimators = [
    ('nn', NN(lr=0.01)),
    ('rf', RandomForestClassifier(max_depth=20, max_features="sqrt", class_weight="balanced",n_estimators=200, n_jobs=40)),
    ('svm', make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True, gamma="auto"))),
    ('hgboost', HistGradientBoostingClassifier(learning_rate=0.1, max_leaf_nodes=63, min_samples_leaf=80, class_weight="balanced"))
]
cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
model=StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1, cv=cv
)
torch.set_num_threads(64)

In [ ]:
display=Display_auc(x,y,model)
display.run()

In [ ]:
print("ROC-AUC:{:.3f}±{:.3f}".format(mean(display.scores_rocauc), stdev(display.scores_rocauc)))
print("PR-AUC:{:.3f}±{:.3f}".format(mean(display.scores_prauc), stdev(display.scores_prauc)))
print("MCC:{:.3f}±{:.3f}".format(mean(display.scores_mcc), stdev(display.scores_mcc)))

In [ ]:
model=make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True, gamma="auto"))
svm_display=Display_auc(x,y,model)
svm_display.run()

In [ ]:
print("ROC-AUC:{:.3f}±{:.3f}".format(mean(svm_display.scores_rocauc), stdev(svm_display.scores_rocauc)))
print("PR-AUC:{:.3f}±{:.3f}".format(mean(svm_display.scores_prauc), stdev(svm_display.scores_prauc)))
print("MCC:{:.3f}±{:.3f}".format(mean(svm_display.scores_mcc), stdev(svm_display.scores_mcc)))

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)

fpr = np.linspace(0, 1, 1000)
nda=np.array(svm_display.tprs)
for i, tpr in enumerate(nda):
    ax.plot(
    fpr,
    tpr,
    color="orange",
    lw=1,
    alpha=0.2
    )
mean_tpr = np.mean(nda, axis=0)
mean_tpr[-1] = 1.0
mean_auc=mean(svm_display.scores_rocauc)
std_auc=stdev(svm_display.scores_rocauc)
ax.plot(
    fpr,
    mean_tpr,
    color="orangered",
    label=r"SVM (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=1,
)

fpr = np.linspace(0, 1, 1000)
nda=np.array(display.tprs)
for i, tpr in enumerate(nda):
    ax.plot(
    fpr,
    tpr,
    color="b",
    lw=1,
    alpha=0.2
    )
mean_tpr = np.mean(nda, axis=0)
mean_tpr[-1] = 1.0
mean_auc=mean(display.scores_rocauc)
std_auc=stdev(display.scores_rocauc)
ax.plot(
    fpr,
    mean_tpr,
    color="b",
    label=r"Stacked model (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", alpha=0.3)
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05]
)


ax.set_xlabel("False positive rate", fontsize=13)
ax.set_ylabel("True positive rate", fontsize=13)
ax.set_title("Client vs. Non-LLPS", fontsize=15)
handles, labels = ax.get_legend_handles_labels()
order = [1, 0]
ax.legend([handles[idx] for idx in order], [labels[idx] for idx in order], loc="lower right", fontsize=10)

plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)

all_precision = []
mean_recall = np.linspace(0, 1, 1000)
for precision, recall in zip(svm_display.precisions, svm_display.recalls):
    ax.plot(
    recall,
    precision,
    color="orange",
    lw=1,
    alpha=0.2
    )
    interp_precision = interp1d(recall, precision)
    resampled_precision = interp_precision(mean_recall)
    all_precision.append(resampled_precision)
mean_precision = np.mean(all_precision, axis=0)
mean_auc=mean(svm_display.scores_prauc)
std_auc=stdev(svm_display.scores_prauc)
ax.plot(
    mean_recall,
    mean_precision,
    color="orangered",
    label=r"SVM (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=1,
)

all_precision = []
mean_recall = np.linspace(0, 1, 1000)
for precision, recall in zip(display.precisions, display.recalls):
    ax.plot(
    recall,
    precision,
    color="b",
    lw=1,
    alpha=0.2
    )
    interp_precision = interp1d(recall, precision)
    resampled_precision = interp_precision(mean_recall)
    all_precision.append(resampled_precision)
mean_precision = np.mean(all_precision, axis=0)
mean_auc=mean(display.scores_prauc)
std_auc=stdev(display.scores_prauc)
ax.plot(
    mean_recall,
    mean_precision,
    color="b",
    label=r"Stacked model (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05]
)


ax.set_xlabel("Recall", fontsize=13)
ax.set_ylabel("Precision", fontsize=13)
ax.set_title("Client vs. Non-LLPS", fontsize=15)

handles, labels = ax.get_legend_handles_labels()
order = [1, 0]
ax.legend([handles[idx] for idx in order], [labels[idx] for idx in order], loc="lower right", fontsize=10)

plt.show()

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
w, p = wilcoxon(display.scores_prauc, svm_display.scores_prauc)

print(f"W-statistic: {w}")
print(f"P-value: {p}")